In [1]:
# command.py
# I'll use this file to turn arrays of numbers
# into design offsets in tao

# Library imports:

# standard imports:
import numpy as np
from numpy import pi
import numpy.linalg as la
import numpy.random as rd
import secrets

import scipy as sp
from scipy.integrate import quad
import pandas as pd
import matplotlib.pyplot as plt

# accelerator imports
from pytao import Tao


In [22]:

# tao = Tao('-init tao.init')

tao.cmd('exit')

['[INFO 2023-JUL-26 16:34:12] tao_command:', '    Stopping.']

In [ ]:
# tao.cmd('exit')
tao.cmd('sho var hfld')

In [ ]:
# for u in range(1,7):
#     modmat = np.zeros((10,12))
#     desmat = np.zeros((10,12))
#     tao.cmd('view '+str(u))
#     for i in range(10):
        
#         tao.cmd('set var hfld['+str(3*i+1)+':'+str(3*i+3)+']|model = 0.00001')
#         for j in range(1,13):
#             if j == 11:
#                 modmat[i][j-1] = float(tao.cmd('python data_parameter '+str(u)+'@orbit.xval[77] model_value')[0].lstrip('1;'))
#                 desmat[i][j-1] = float(tao.cmd('python data_parameter '+str(u)+'@orbit.xval[77] design_value')[0].lstrip('1;'))
#             elif j == 12:
#                 modmat[i][j-1] = float(tao.cmd('python data_parameter '+str(u)+'@orbit.xval[78] model_value')[0].lstrip('1;'))
#                 desmat[i][j-1] = float(tao.cmd('python data_parameter '+str(u)+'@orbit.xval[78] design_value')[0].lstrip('1;'))
#             else:
#                 modmat[i][j-1] = float(tao.cmd('python data_parameter '+str(u)+'@orbit.xval['+str(j)+'] model_value')[0].lstrip('1;'))
#                 desmat[i][j-1] = float(tao.cmd('python data_parameter '+str(u)+'@orbit.xval['+str(j)+'] design_value')[0].lstrip('1;'))
#         tao.cmd('set var hfld['+str(3*i+1)+':'+str(3*i+3)+']|model = 0')
        
#     Rmat = modmat.T-desmat.T
#     np.savetxt('/Users/alexcoxe/My Drive/code/thesis?/FFA@CEBAF tests/SVDtests/SVD Response Matrices/#c=10_x-only/wp'+str(u)+'Rmat_e-5.dat',Rmat)
#     U, S, Vh = la.svd(Rmat)
#     # print(Rmat)
#     np.savetxt('/Users/alexcoxe/My Drive/code/thesis?/FFA@CEBAF tests/SVDtests/SVD Response Matrices/#c=10_x-only/wp'+str(u)+'_svdU_e-5.dat',U)
#     np.savetxt('/Users/alexcoxe/My Drive/code/thesis?/FFA@CEBAF tests/SVDtests/SVD Response Matrices/#c=10_x-only/wp'+str(u)+'_svdS_e-5.dat',S)
#     np.savetxt('/Users/alexcoxe/My Drive/code/thesis?/FFA@CEBAF tests/SVDtests/SVD Response Matrices/#c=10_x-only/wp'+str(u)+'svdVh_e-5.dat',Vh)
#     # print('Universe',u)
#     # print(Rmat)
#     # print('[U] ='+str(U.shape)+': [S] ='+str(S.shape)+': [Vh] =',str(Vh.shape))
#     # print('[(U_S_Vh)] =',usvh.shape)


In [13]:
# I'm gonna test this decomposition on just the first universe

modmat = np.zeros((3,5))
desmat = np.zeros((3,5))

for i in range(3):
        
    tao.cmd('set var hfld['+str(i+1)+']|model = 1', raises=False)
    for j in range(5):
        if j == 3:
            modmat[i][j] = float(tao.cmd('python data_parameter '+str(1)+'@orbit.xval[77] model_value')[0].lstrip('1;'))
            desmat[i][j] = float(tao.cmd('python data_parameter '+str(1)+'@orbit.xval[77] design_value')[0].lstrip('1;'))
        elif j == 4:
            modmat[i][j] = float(tao.cmd('python data_parameter '+str(1)+'@orbit.xval[78] model_value')[0].lstrip('1;'))
            desmat[i][j] = float(tao.cmd('python data_parameter '+str(1)+'@orbit.xval[78] design_value')[0].lstrip('1;'))
        else:
            modmat[i][j] = float(tao.cmd('python data_parameter '+str(1)+'@orbit.xval['+str(j+1)+'] model_value')[0].lstrip('1;'))
            desmat[i][j] = float(tao.cmd('python data_parameter '+str(1)+'@orbit.xval['+str(j+1)+'] design_value')[0].lstrip('1;'))
    
    tao.cmd('set var hfld['+str(i+1)+']|model = 0', raises=False)
        
Rmat = (desmat.T-modmat.T)
# print(Rmat.shape)
U, S, Vh = la.svd(Rmat)
    # print('Universe',u)
    # print(Rmat)
    # print('[U] ='+str(U.shape)+': [S] ='+str(S.shape)+': [Vh] =',str(Vh.shape))
    # print('[(U_S_Vh)] =',usvh.shape)
print(S)

[0.00095412 0.00068715 0.00020122]


In [14]:
xdes = list(map(float,tao.cmd('python data_parameter 1@orbit.xval[1:3,77,78] design_value')[0].lstrip('1;').split(';')))
# xendval = float(tao.cmd('python data_parameter 1@orbit.xval[77] design_value')[0].lstrip('1;'))
# xendtot = float(tao.cmd('python data_parameter 1@orbit.xval[78] design_value')[0].lstrip('1;'))
# # print('shape of bpm readings = ',np.shape(xdes))
# print(xendval)
# print(xendtot,'\n')
# xdes = np.append(xdes,[xendval,xendtot])
# print('shape of bpm readings = ',np.shape(xdes))
xdes = np.reshape(xdes, (5,1))
print(xdes)

[[-0.02099588]
 [-0.02099588]
 [-0.02099588]
 [-0.02099588]
 [ 0.02248383]]


In [15]:

tao.cmd('set ele BFMG##1 x_offset = 0.00005', raises=False)
xmod1 = np.reshape(list(map(float,tao.cmd('python data_parameter 1@orbit.xval[1:3,77,78] model_value')[0].lstrip('1;').split(';'))),(5,1))
# xmod = np.reshape(
#     np.append(list(map(float,tao.cmd('python data_parameter 1@orbit.xval[3:10] model_value')[0].lstrip('1;').split(';'))),
#           [float(tao.cmd('python data_parameter 1@orbit.xval[77] model_value')[0].lstrip('1;')),
#            float(tao.cmd('python data_parameter 1@orbit.xval[78] model_value')[0].lstrip('1;'))]),
#            (10,1))
dx = xmod1 - xdes
print(dx)

[[ 0.0000000e+00]
 [ 6.8724560e-05]
 [ 1.2867102e-04]
 [-1.0304714e-04]
 [ 3.0647747e-04]]


In [16]:

tao.cmd('veto var vfld',raises=False)
tao.cmd('veto var b1corr',raises=False)


['  hfld[1:25]                                     Using: 1:25',
 '  vfld[1:25]                                     Using:',
 '  b1corr[1:25]                                   Using:',
 '[FATAL | 2023-JUL-26 14:44:20] tao_pointer_to_universe_int:',
 '    UNIVERSE INDEX OUT OF RANGE: 6']

In [21]:
tao.cmd('run svd', raises=False)

['Optimizing with: svd',
 "Type ``.'' to stop the optimizer before it's finished.",
 '[INFO] tao_dmodel_dvar_calc:',
 '    Remaking dModel_dVar derivative matrix.',
 '    This may take a while...',
 '',
 'Hit upper limit of variable: hfld[4]',
 '[INFO] tao_veto_vars_with_zero_dmodel:',
 '    Data is independent of Variable: hfld[4]',
 '    Will not use this variable in optimization.',
 'Initial Merit:     1.8351E+03',
 'Hit upper limit of variable: hfld[1]',
 'Hit upper limit of variable: hfld[2]',
 'Hit upper limit of variable: hfld[3]',
 'Hit lower limit of variable: hfld[5]',
 'Hit lower limit of variable: hfld[6]',
 'Hit upper limit of variable: hfld[7]',
 'Hit upper limit of variable: hfld[8]',
 'Hit lower limit of variable: hfld[9]',
 'Hit lower limit of variable: hfld[10]',
 'Hit lower limit of variable: hfld[11]',
 'Hit upper limit of variable: hfld[12]',
 'Hit lower limit of variable: hfld[13]',
 'Hit lower limit of variable: hfld[14]',
 'Hit lower limit of variable: hfld[15]'

In [18]:
Sr = np.zeros((3,5))
for i in range(len(S)):
    if S[i] < np.power(10.,-20): Sr[i][i] = 0
    else: Sr[i][i] = 1/S[i]
print(np.diagonal(Sr))
# print(S)

[1048.08379658 1455.29374036 4969.67566652]


In [19]:
dmag=np.reshape(np.dot(np.dot(Vh.T,np.dot(Sr,U.T)),xmod1),(3,1))
print(dmag)

[[-72.10489052]
 [-58.16180259]
 [100.75776949]]


In [ ]:
for i in range(5):
    if dmag[i] > 0.04:
        tao.cmd('set var hfld['+str(i+1)+']|model = '+str(0.04), raises=False)
    if dmag[i] < -0.04:
        tao.cmd('set var hfld['+str(i+1)+']|model = '+str(-0.04), raises=False)
    else:
        tao.cmd('set var hfld['+str(i+1)+']|model = '+str(dmag[i]), raises=False)

In [20]:
xmod2 = np.reshape(list(map(float,tao.cmd('python data_parameter 1@orbit.xval[1:3,77,78] model_value')[0].lstrip('1;').split(';'))),(5,1))
# xmod2 = np.reshape(
#     np.append(list(map(float,tao.cmd('python data_parameter 1@orbit.xval[3:10] model_value')[0].lstrip('1;').split(';'))),
#           [float(tao.cmd('python data_parameter 1@orbit.xval[77] model_value')[0].lstrip('1;')),
#            float(tao.cmd('python data_parameter 1@orbit.xval[78] model_value')[0].lstrip('1;'))]),
#            (10,1))
dx2 = xmod2 - xdes
print(dx2)

[[ 0.0000000e+00]
 [ 6.8724560e-05]
 [ 1.2867102e-04]
 [-1.0304714e-04]
 [ 3.0647747e-04]]


In [ ]:
fmod=float(tao.cmd('python data_parameter 1@orbit.xval[78] model_value')[0].lstrip('1;'))
fdes=float(tao.cmd('python data_parameter 1@orbit.xval[78] design_value')[0].lstrip('1;'))
print(180*(fmod-fdes)/pi)

In [ ]:
pm = float(tao.cmd('python data_parameter 1@orbit.pxval[76] model_value')[0].lstrip('1;'))
pd = float(tao.cmd('python data_parameter 1@orbit.pxval[76] design_value')[0].lstrip('1;'))

# thm = float(tao.cmd('python data_parameter 1@floor.theta model_value')[0].lstrip('1;'))
# thd = float(tao.cmd('python data_parameter 1@floor.theta design_value')[0].lstrip('1;'))

print('design px =',str(pd)+': model px =',str(pm))
# print('design floor.theta =',str(thd)+': model floor.theta =',str(thm))